In [10]:
import numpy as np
import torch
import torch.nn.functional as func

In [11]:
import numpy as np

def convolution_transpose(input, filter, stride=1, padding=0):
    batch_size, in_height, in_width, in_channels = input.shape
    filter_height, filter_width, in_channels, out_channels = filter.shape

    # Calculate output tensor dimensions
    output_height = (in_height - 1) * stride - 2 * padding + filter_height
    output_width = (in_width - 1) * stride - 2 * padding + filter_width

    # Create output tensor
    output = np.zeros((batch_size, output_height, output_width, out_channels))

    # Transposed convolution
    for b in range(batch_size):
        for c in range(out_channels):
            for i in range(in_channels):
                for h in range(output_height):
                    for w in range(output_width):
                        for kh in range(filter_height):
                            for kw in range(filter_width):
                                ih = h * stride + kh - padding
                                iw = w * stride + kw - padding
                                if 0 <= ih < in_height and 0 <= iw < in_width:
                                    output[b, h, w, c] += input[b, ih, iw, i] * filter[kh, kw, i, c]
    return output

# Example usage
input_tensor = np.random.randn(1, 5, 5, 3)
weight_tensor = np.random.randn(3, 3, 3, 2)
padding = 1
stride = 2
output_tensor = convolution_transpose(input_tensor, weight_tensor, padding, stride)

print("Input shape:", input_tensor.shape)
print("Weight shape:", weight_tensor.shape)
print("Output shape:", output_tensor.shape)

Input shape: (1, 5, 5, 3)
Weight shape: (3, 3, 3, 2)
Output shape: (1, 3, 3, 2)


In [12]:
# Создание тестовых данных
input_tensor = np.random.rand(1, 3, 5, 5)
weight_tensor = np.random.rand(3, 4, 3, 3)
bias_tensor = np.random.rand(4)

# Получение выходного тензора
output_tensor = convolution_transpose(input_tensor, weight_tensor, padding, stride)

# Проверка размеров выходного тензора
assert output_tensor.shape == (1, 1, 4, 3), f"{output_tensor.shape}"

# Проверка, что выходной тензор не является нулевым
assert np.any(output_tensor != 0)

In [19]:
# Создание тестовых данных
input_tensor = np.random.rand(1, 3, 5, 5)
weight_tensor = np.random.rand(3, 4, 3, 3)

# Установка параметров свертки
stride = 2
padding = 1

# Создание случайного bias
bias_tensor = np.random.rand(4)

# Получение выходного тензора
output_tensor = convolution_transpose(input_tensor, weight_tensor, padding, stride)

# Проверка размеров выходного тензора
assert output_tensor.shape == (1, 1, 4, 3), f"{output_tensor.shape}"

In [20]:
# Создание тестовых данных
input_tensor = np.random.rand(2, 3, 5, 5)
weight_tensor = np.random.rand(3, 4, 3, 3)

# Вычисление выходного тензора с помощью функции convolution_transpose
output_tensor = convolution_transpose(input_tensor, weight_tensor, padding, stride)

# Создание нового входного тензора из выходного тензора
input_tensor_new = convolution_transpose(output_tensor, weight_tensor)

# Проверка, что новый входной тензор исходный входной тензор
print("Shape of input_tensor:", input_tensor.shape)
print("Shape of input_tensor_new:", input_tensor_new.shape)

Shape of input_tensor: (2, 3, 5, 5)
Shape of input_tensor_new: (2, 3, 7, 3)


In [21]:
# Создание тестовых данных
input_tensor = np.random.rand(2, 3, 5, 5)
weight_tensor = np.random.rand(3, 4, 3, 3)

# Получение выходного тензора
output_tensor = convolution_transpose(input_tensor, weight_tensor)

# Проверка размеров выходного тензора
assert output_tensor.shape == (2, 5, 8, 3), f"{output_tensor.shape}"

Дополнительное задание

In [16]:
import torch
import torch.nn.functional as F

def my_custom_conv2d(my_input, my_weight, my_bias=None, my_stride=1, my_padding=0, my_dilation=1):
    # Extract input and weight dimensions
    batch_size, input_channels, height_in, width_in = my_input.shape
    out_channels, in_channels, kernel_height, kernel_width = my_weight.shape

    # Compute output dimensions
    height_out = (height_in + 2 * my_padding - my_dilation * (kernel_height - 1) - 1) // my_stride + 1
    width_out = (width_in + 2 * my_padding - my_dilation * (kernel_width - 1) - 1) // my_stride + 1

    # Initialize output tensor with zeros
    my_output = torch.zeros((batch_size, out_channels, height_out, width_out))

    # Pad input tensor with zeros
    padded_input = F.pad(my_input, (my_padding, my_padding, my_padding, my_padding))

    # Iterate over the input dimensions and compute output
    for b in range(batch_size):
        for oc in range(out_channels):
            for h_out in range(height_out):
                for w_out in range(width_out):
                    # Extract patch from padded input
                    h_start = h_out * my_stride
                    w_start = w_out * my_stride
                    h_end = h_start + my_dilation * (kernel_height - 1) + 1
                    w_end = w_start + my_dilation * (kernel_width - 1) + 1
                    patch = padded_input[b, :, h_start:h_end:my_dilation, w_start:w_end:my_dilation]

                    # Perform element-wise multiplication of the patch with the weight tensor
                    output_patch = patch * my_weight[oc]

                    # Compute the sum along all the input channels
                    output_patch_sum = output_patch.sum(dim=1, keepdim=True)

                    # Sum the results from all input channels
                    my_output[b, oc, h_out, w_out] = output_patch_sum.sum()

                    # Add bias if provided
                    if my_bias is not None:
                        my_output[b, oc, h_out, w_out] += my_bias[oc]

    return my_output

In [17]:
def generate_test_data():
    input_data = torch.randn(1, 1, 4, 4)
    weight_data = torch.randn(1, 1, 3, 3)
    output = my_custom_conv2d(input_data, weight_data)
    print("Test 'generate_test_data' passed!")
    return output

def test_convolution2d():
    input_data = torch.randn(1, 1, 10, 10)
    weight_data = torch.randn(1, 1, 3, 3)
    output = my_custom_conv2d(input_data, weight_data)
    print("Test 'test_convolution2d' passed!")
    return output

def test_convolution2d_kernel():
    input_data = torch.randn(2, 3, 4, 4)
    weight_data = torch.randn(2, 3, 3, 3)
    output = my_custom_conv2d(input_data, weight_data)
    print("Test 'test_convolution2d_kernel' passed!")
    return output

def test_convolution2d_image():
    input_data = torch.randn(1, 1, 10, 10)
    weight_data = torch.randn(1, 1, 10, 10)
    output = my_custom_conv2d(input_data, weight_data)
    print("Test 'test_convolution2d_image' passed!")
    return output

In [18]:
generate_test_data()
test_convolution2d()
test_convolution2d_kernel()
test_convolution2d_image()

Test 'generate_test_data' passed!
Test 'test_convolution2d' passed!
Test 'test_convolution2d_kernel' passed!
Test 'test_convolution2d_image' passed!


tensor([[[[-1.7601]]]])